In [2]:
import time
import numpy as np
import pandas as pd
import optunity
import optunity.metrics

from sklearnex import patch_sklearn 
patch_sklearn()

from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization

from Function import svm_function

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [10]:
X = np.load("data/merge_data/seq_rbp/transcript2pse_in_one*rbp_pvalue_cutoff10-3_-log10__train__x.npy")
feature_names = pd.read_csv("data/merge_data/ALL/transcript2pse_in_one*rbp_pvalue__cutoff10-3.csv").columns.drop(["loc", "Transcript_stable_ID"])

# X = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__x.npy")
# feature_names = pd.read_csv("data/merge_data/ALL/transcript2pse_in_one2rbp_pvalue_cutoff10-3_-log10.csv").columns.drop(["loc", "Transcript_stable_ID"])

y = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy")

In [7]:
u_test_pval = np.load("data/merge_data/seq_rbp/t2p*r_p_10-3_-log10__trian__x__utest_pval.npy")

In [11]:
X_temp = X.copy()

In [13]:
X = X_temp[:, u_test_pval < 0.05]
print(X_temp.shape)

(2264, 21168)
(2264, 4665)


In [14]:
X = X_temp[:, u_test_pval < 0.01]
print(X.shape)

(2264, 2704)


In [15]:
print(X.shape)
np.save("data/merge_data/seq_rbp/pse_in_one*rbp_pvalue_cutoff10-3_-log10__utest_pval001__train__x.npy", X)

(2264, 2704)


In [5]:
fold = 10

cv_x, cv_y = svm_function.CV_balanced(X, y, fold)

auroc_array = []
feature_importances_array = []
for i in range(fold):
    x_train, y_train, x_test, y_test = svm_function.cv_train_test(cv_x, cv_y, i)
    clf = RandomForestClassifier(n_jobs=-1)
    clf.fit(x_train, y_train)
    
    feature_importances_array.append(clf.feature_importances_)
    
    y_test_score = clf.predict_proba(x_test)[:, 1]
    roc_score = metrics.roc_auc_score(y_test, y_test_score)
    auroc_array.append(roc_score)

In [6]:
auroc_array

[0.7230769230769231,
 0.7073732718894009,
 0.662606978275181,
 0.6783760683760683,
 0.6092001316655694,
 0.7181533903884134,
 0.72,
 0.527238314680711,
 0.689680710994075,
 0.6385780118499013]

In [5]:
cv_x, cv_y = svm_function.CV_balanced(X[:, np.sum(np.array(feature_importances_array), axis=0) > 0], y, fold)

temp_auroc_array = []
temp_feature_importances_array = []
for i in range(fold):
    x_train, y_train, x_test, y_test = svm_function.cv_train_test(cv_x, cv_y, i)
    clf = RandomForestClassifier(n_jobs=-1)
    clf.fit(x_train, y_train)
    
    temp_feature_importances_array.append(clf.feature_importances_)
    
    y_test_score = clf.predict_proba(x_test)[:, 1]
    roc_score = metrics.roc_auc_score(y_test, y_test_score)
    temp_auroc_array.append(roc_score)

In [7]:
print(sum(temp_auroc_array) / len(temp_auroc_array))
print(sum(auroc_array) / len(auroc_array))

0.6610937554508983
0.6381885996747748


In [7]:
print(sum(np.sum(np.array(feature_importances_array), axis=0) > 0))
print(sum(np.sum(np.array(temp_feature_importances_array), axis=0) > 0))

4507


NameError: name 'temp_feature_importances_array' is not defined

In [24]:
print(sum(np.sum(np.array(feature_importances_array) != 0, axis=0) > 5))
print(sum(np.sum(np.array(temp_feature_importances_array) != 0, axis=0) > 5))

7267
7402


In [36]:
print(sum(np.sum(np.array(feature_importances_array) > np.average(np.array(feature_importances_array)), axis=0) != 0))
print(sum(np.sum(np.array(temp_feature_importances_array) > np.average(np.array(temp_feature_importances_array)), axis=0) > 5))

13168
2557


In [26]:
array = 1 - np.array(clf.feature_importances_)
# array = np.array([1,1,5, 0, 7, 0.5])
order = array.argsort()
ranks = order.argsort()

In [23]:
def get_params(x):
    C_Nu = "Nu" if x[0] > 0.5 else "C"
    kernel_list = ['linear',
                    'poly', 
                    'rbf', 
                    'sigmoid']
    kernel = kernel_list[int(x[1] * 3)]
    kernel = "%s_%s" % (C_Nu, kernel)
    C = (20 * x[2]) - 10
    logGamma = (20 * x[3]) - 10
    degree = int(x[4] * 10)
    coef0 = (20 * x[5]) - 10
    n = x[6] if x[6] != 0 else 1e-7
    
    params = {
        'kernel':kernel, 
        'C':C, 
        'logGamma':logGamma, 
        'degree':degree, 
        'coef0':coef0, 
        'n':n, 
    }
    return params

class eSVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99, size=1, max_iter=1000, fold=5):
        super().__init__(dimension=X_train.shape[1] + 7, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha
        self.size = size
        self.max_iter = max_iter
        self.fold = fold
        
    def _evaluate(self, x):
        params = get_params(x[:7])
        
        x = x[7:]
        selected = x > 0.5
        num_selected = selected.sum()
        print(params)
        if num_selected == 0:
            # auroc = 
            return 1
        else:
            # auroc = 1
            # auroc = cv_mp_esvm(self.X_train[:, selected], self.y_train, fold=self.fold, size=self.size, max_iter=self.max_iter,
            #             **params 
            #             )['avg AUROC']
            auroc = svm_function.cv_esvm_perf(self.X_train[:, selected], self.y_train, fold=self.fold, size=self.size, max_iter=self.max_iter,
                        **params 
                        )['avg AUROC']
        print("auroc %.4f, avg %.4f, std %.4f, selected %d" % (auroc, sum(x) / len(x), np.std(x), num_selected))
        print("socre=%.4f" % (self.alpha * (1 - auroc) + (1 - self.alpha) * (num_selected / self.X_train.shape[1])))
        return self.alpha * (1 - auroc) + (1 - (self.alpha)) * (num_selected / self.X_train.shape[1])
        # return self.alpha * (1 - auroc) + (1 - (self.alpha / 2)) * (num_selected / self.X_train.shape[1]) + (1 - (self.alpha / 2)) * (sum(x) / len(x))

class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99, max_iter=1000, fold=5):
        super().__init__(dimension=X_train.shape[1] + 7, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha
        self.max_iter = max_iter
        self.fold = fold
        
    def _evaluate(self, x):
        params = get_params(x[:7])
        
        x = x[7:]
        selected = x > 0.5
        num_selected = selected.sum()
        print(params)
        if num_selected == 0:
            return 1
        else:
            try:
                auroc = svm_function.cv_svm_perf(self.X_train[:, selected], self.y_train, fold=self.fold, max_iter=self.max_iter,
                            **params 
                            )['avg AUROC']
            except:
                auroc = 0.5
        print("auroc %.4f, avg %.4f, std %.4f, selected %d" % (auroc, sum(x) / len(x), np.std(x), num_selected))
        print("socre=%.4f" % (self.alpha * (1 - auroc) + (1 - self.alpha) * (num_selected / self.X_train.shape[1])))
        return self.alpha * (1 - auroc) + (1 - (self.alpha)) * (num_selected / self.X_train.shape[1])
  

In [6]:
total_time = time.time()
size = 1
max_iter = 1000
fold = 5

problem = eSVMFeatureSelection(X[:, np.sum(np.array(feature_importances_array), axis=0) > 0], y, size=size, max_iter=max_iter, fold=fold, alpha=0.8)

task = Task(problem, max_iters=50)
algorithm = ParticleSwarmOptimization(population_size=10000, seed=1234)
best_features, best_fitness = algorithm.run(task)
params = get_params(best_features[:7])
params['size'] = size
params['fold'] = fold
params['max_iter'] = max_iter

selected_features = best_features[7:] > 0.5

# print("subset eSVM train:")
# subset_perf = svm_function.cv_esvm_perf(X[:, np.sum(np.array(feature_importances_array), axis=0) > 0][:, selected_features], y, **params)


print("params:")
print(params)
print('Number of selected features: %s / %s' % (selected_features.sum(), X[:, np.sum(np.array(feature_importances_array), axis=0) > 0].shape[1]))
print("best_fitness: %s" % best_fitness)
# print("Subset roc_score: %s" % subset_perf['avg AUROC'])

# total_time = time.time() - total_time
# json_dict = {
#     # "all_perf": all_perf,
#     "total_time": total_time,
#     "subset_perf": subset_perf,
#     "params": params,
#     "selected_features": list(selected_features.astype(str)),
# }
# print(json_dict)

NameError: name 'feature_importances_array' is not defined

In [ ]:
total_time = time.time()
size = 1
max_iter = 1000
fold = 5

problem = eSVMFeatureSelection(X, y, size=size, max_iter=max_iter, fold=fold, alpha=0.5)

task = Task(problem, max_iters=50)
algorithm = ParticleSwarmOptimization(population_size=4000, seed=1234)
best_features, best_fitness = algorithm.run(task)
params = get_params(best_features[:7])
params['size'] = size
params['fold'] = fold
params['max_iter'] = max_iter

selected_features = best_features[7:] > 0.5

print("params:")
print(params)
print('Number of selected features: %s / %s' % (selected_features.sum(), X.shape[1]))
print("best_fitness: %s" % best_fitness)

---

In [25]:
total_time = time.time()
max_iter = 1000
fold = 5

problem = SVMFeatureSelection(X, y, max_iter=max_iter, fold=fold, alpha=0.5)

task = Task(problem, max_iters=50)
algorithm = ParticleSwarmOptimization(population_size=4000, seed=1234)
best_features, best_fitness = algorithm.run(task)
params = get_params(best_features[:7])
params['fold'] = fold
params['max_iter'] = max_iter

selected_features = best_features[7:] > 0.5

print("params:")
print(params)
print('Number of selected features: %s / %s' % (selected_features.sum(), X.shape[1]))
print("best_fitness: %s" % best_fitness)

In [ ]:

esvm_space = {'kernel': {
                    'C_linear': {'C': [-10, 10]},
                    'C_poly': {'logGamma': [-10, 10], 'C': [-10, 10], 'degree': [1, 10], 'coef0': [-10, 10]},
                    'C_rbf': {'logGamma': [-10, 10], 'C': [-10, 10]},
                    'C_sigmoid': {'logGamma': [-10, 10], 'C': [-10, 10], 'coef0': [-10, 10]},
                    'Nu_linear': {'n': [0, 1]},
                    'Nu_poly': {'logGamma': [-10, 10], 'n': [0, 1], 'degree': [1, 10], 'coef0': [-10, 10]},
                    'Nu_rbf': {'logGamma': [-10, 10], 'n': [0, 1]},
                    'Nu_sigmoid': {'logGamma': [-10, 10], 'n': [0, 1], 'coef0': [-10, 10]}
                    }
        }

input_file = "data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__x.npy"
label_file = "data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy"
print("Input file: %s" % (input_file))
data_x = np.load(input_file)
print("Label file: %s" % (label_file))
data_y = np.load(label_file)

max_iter = 1
size = 1
fold = 5
seed = 1212
pmap = optunity.parallel.create_pmap(1)

In [ ]:
def esvm_tuned_auroc(x_train, y_train, x_test, y_test, kernel='C_linear', C=0, logGamma=0, degree=0, coef0=0, n=0.5, size=size, max_iter=max_iter, **params):
    model = svm_function.esvm_train_model(x_train, y_train, kernel=kernel, C=C, logGamma=logGamma, degree=degree, coef0=coef0, n=n, size=size, max_iter=max_iter)
    roc_score = model.test(x_test, y_test)
    print("AUROC: %.2f" % roc_score)
    return roc_score

cv_decorator = optunity.cross_validated(x=data_x, y=data_y, num_folds=fold)
cv_esvm_tuned_auroc = cv_decorator(esvm_tuned_auroc)
optimal_svm_pars, info, _ = optunity.maximize_structured(cv_esvm_tuned_auroc, esvm_space, num_evals=args.num_evals, pmap=pmap)
print("optunity done.")

In [ ]:
import time
import tqdm
import multiprocessing

from sklearn import metrics
from itertools import repeat

class multi_ensemble_svm():
    def __init__(self):
        self.model_array = None
        self.model_size = None
        self.hp = {
            "kernel": 'C_linear', "C":0, "logGamma":0, "degree":0, "coef0":0, "n":0.5, "max_iter":1e7
        }
    
    def train(self, data, label, ensemble_data_size=1, kernel='C_linear', C=0, logGamma=0, degree=0, coef0=0, n=0.5, max_iter=1e7):
        train_time = time.time()
        self.hp["kernel"] = kernel
        self.hp["C"] = C
        self.hp["logGamma"] = logGamma
        self.hp["degree"] = degree
        self.hp["coef0"] = coef0
        self.hp["n"] = n
        self.hp["max_iter"] = max_iter
        
        x, y = svm_function.ensemble_data(data, label, size=ensemble_data_size)
        self.model_size = len(x)
        
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        result = pool.starmap(self._svm_train, tqdm.tqdm(zip(x, y), total=self.model_size))
        self.model_array = result
        print("ensemble svm train: %.2fs" % (time.time() - train_time))
        pool.close()
        return None

    def _svm_train(self, d, l):
        arr = np.arange(len(l))
        np.random.shuffle(arr)
        d = d[arr]
        l = l[arr]
        
        m = svm_function.svm_train_model(d, l, self.hp["kernel"], self.hp["C"], self.hp["logGamma"], self.hp["degree"], self.hp["coef0"], self.hp["n"], max_iter=self.hp["max_iter"])
        return m
    
    def test(self, x, y):
        pred_y, pred_y_score = self.predict(x)
            
        return metrics.roc_auc_score(y, pred_y_score)
    
    def _svm_predict(self, m, x):
        return m.predict(x)
    
    def predict(self, x):
        output = None
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        output = pool.starmap(self._svm_predict, tqdm.tqdm(zip(self.model_array, repeat(x)), total=self.model_size))
        pred_y_score = np.sum(output, axis=0) / self.model_size
        pred_y = np.where(pred_y_score >= 0.5, 1, 0)
        
        pool.close()
        return pred_y, pred_y_score

def cv_mp_esvm(data_x, data_y, fold=5,
                kernel='C_rbf', 
                C=0, 
                logGamma=0, 
                degree=3, 
                coef0=0, 
                n=0, 
                size=10, 
                max_iter=1000):
    cv_x, cv_y = svm_function.CV_balanced(data_x, data_y, fold)
        
    auroc_array = []
    for i in range(fold):
        x_train, y_train, x_test, y_test = svm_function.cv_train_test(cv_x, cv_y, i)
        clf = multi_ensemble_svm()
        clf.train(x_train, y_train, ensemble_data_size=size, kernel=kernel, 
                C=C, 
                logGamma=logGamma, 
                degree=degree, 
                coef0=coef0, 
                n=n, 
                max_iter=max_iter)
        auroc_array.append(clf.test(x_test, y_test))
    return sum(auroc_array) / len(auroc_array)

In [ ]:
X = np.load("data/merge_data/seq_rbp/transcript2pse_in_one*rbp_pvalue_cutoff10-3_-log10__train__x.npy")
y = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy")
feature_names = pd.read_csv("data/merge_data/ALL/transcript2pse_in_one*rbp_pvalue__cutoff10-3.csv").columns.drop(["loc", "Transcript_stable_ID"])

In [ ]:
X = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__x.npy")
y = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy")
feature_names = pd.read_csv("data/merge_data/ALL/transcript2pse_in_one2rbp_pvalue_cutoff10-3_-log10.csv").columns.drop(["loc", "Transcript_stable_ID"])

In [ ]:
cv_x, cv_y = svm_function.CV_balanced(X, y, 10)

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
np.unique(cv_y[0], return_counts=True)

In [ ]:
cv_x[1].shape

In [ ]:
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
feature_names = dataset.feature_names

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

problem = SVMFeatureSelection(X_train, y_train)

task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)

selected_features = best_features > 0.5
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(feature_names[selected_features].tolist()))

In [ ]:
clf = multi_ensemble_svm()
clf.train(X_train[:, selected_features], y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    ensemble_data_size=10, 
                                    max_iter=1000)
subset_roc_score = clf.test(X_test[:, selected_features], y_test)


clf = multi_ensemble_svm()
clf.train(X_train, y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    ensemble_data_size=10, 
                                    max_iter=1000)
roc_score = clf.test(X_test, y_test)
print('Subset roc_score:', subset_roc_score)
print('All roc_score:', roc_score)

In [ ]:
model = svm_function.esvm_train_model(X_train[:, selected_features], y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    size=10, 
                                    max_iter=1000)
roc_score = model.test(X_test[:, selected_features], y_test)
print('Subset roc_score:', roc_score)

model = svm_function.esvm_train_model(X_train, y_train, kernel='C_rbf', 
                                    C=-0.24806890111201105, 
                                    logGamma=-4.955918931113543, 
                                    degree=3, 
                                    coef0=0, 
                                    n=0, 
                                    size=10, 
                                    max_iter=1000)
roc_score = model.test(X_test, y_test)
print('All roc_score:', roc_score)

In [ ]:
from sklearn import metrics

model_selected = SVC()
model_all = SVC()

model_selected.fit(X_train[:, selected_features], y_train)
decision_values = model_selected.decision_function(X_test[:, selected_features])
roc_score = metrics.roc_auc_score(y_test, decision_values)

print('Subset roc_score:', roc_score)

model_all.fit(X_train, y_train)
decision_values = model_all.decision_function(X_test)
roc_score = metrics.roc_auc_score(y_test, decision_values)
print('All Features roc_score:', roc_score)

In [ ]:
best_features, best_fitness